In [72]:
import json
import re
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
# spacy 
# flashtext

In [69]:
# classification = {'reconnaissance': 'open',
#                  'social_engineering': ['friend', 'family', 'children'],
#                  'malware': ['link', 'click'], 
#                             x['Content-Disp...'] has attachment? 
#                  'credential_phishing': ['SSN', 'social security', 'birth', 'account']}

In [95]:
# just testing
classified_list = []
for i in range(1, 4030):
    classified_dict = {'index': i, 'attack_type':
                       {'reconnaissance': 0, 
                       'social_engineering': 0,
                       'malware': 0,
                       'credential_phishing': 0} 
                       }
    with open(f'../../data/separated_by_email/{i}.json') as json_file:
        x = json.load(json_file)
    if 'dc:subject' in x.keys() and 'X-TIKA:content' in x.keys():
        content = x['dc:subject'] + x['X-TIKA:content'].lower()
        # 1. 
        if x['MboxParser-status'] == 'O':
            classified_dict['attack_type']['reconnaissance'] = 1
        # 2.
        if content.find('urgent') != -1 or content.find('friend') != -1 or content.find('children') != -1 or content.find('family') != -1:
            classified_dict['attack_type']['social_engineering'] = 1
        # 3. 
        if content.find('click') != -1 or content.find('link') != -1:
            classified_dict['attack_type']['malware'] = 1
        # 4. 
        if content.find('SSN') != -1 or content.find('social security') != -1 or content.find('birth') != -1 or content.find('account') != -1:
            classified_dict['attack_type']['credential_phishing'] = 1
    # 3.
    elif 'Content-Disposition' in x.keys():
        if x['Content-Disposition'].find('attachment') != -1:
            classified_dict['attack_type']['malware'] = 1
            
    classified_list.append(classified_dict)

In [99]:
# 5a.
classified_list

[{'index': 1,
  'attack_type': {'reconnaissance': 1,
   'social_engineering': 1,
   'malware': 0,
   'credential_phishing': 1}},
 {'index': 2,
  'attack_type': {'reconnaissance': 1,
   'social_engineering': 1,
   'malware': 0,
   'credential_phishing': 1}},
 {'index': 3,
  'attack_type': {'reconnaissance': 0,
   'social_engineering': 1,
   'malware': 0,
   'credential_phishing': 0}},
 {'index': 4,
  'attack_type': {'reconnaissance': 0,
   'social_engineering': 1,
   'malware': 0,
   'credential_phishing': 0}},
 {'index': 5,
  'attack_type': {'reconnaissance': 0,
   'social_engineering': 1,
   'malware': 0,
   'credential_phishing': 1}},
 {'index': 6,
  'attack_type': {'reconnaissance': 0,
   'social_engineering': 1,
   'malware': 0,
   'credential_phishing': 1}},
 {'index': 7,
  'attack_type': {'reconnaissance': 1,
   'social_engineering': 1,
   'malware': 0,
   'credential_phishing': 1}},
 {'index': 8,
  'attack_type': {'reconnaissance': 1,
   'social_engineering': 1,
   'malware': 0,

In [97]:
for i in tqdm(range(0, 4029)):
    requests.patch(f'https://dsci-550-default-rtdb.firebaseio.com/assignment-1/task-5-additional-features/{i+1}.json', json=classified_list[i])

  0%|          | 0/4029 [00:00<?, ?it/s]

# iv.

In [62]:
no_content = []
has_offer = []
iv_list = []
for i in range(1, 4030):
    iv_dict = {'index': i, 'has_offer': 'no',}
    with open(f'../../data/separated_by_email/{i}.json') as json_file:
        x = json.load(json_file)
    try:
        content = x['X-TIKA:content'].lower()
        if content.find('offer') != -1 or content.find('reward') != -1:
            has_offer.append(i)
            if content.find('service') != -1:
                iv_dict['has_offer'] = 'service'
            elif re.search(r'\d+\%', content) or re.search(r'\$\d+', content):
                iv_dict['has_offer'] = 'money'        
    except: 
        no_content.append(i)
    iv_list.append(iv_dict)

In [63]:
len(has_offer)

828

In [12]:
# just showing the sentences for testing
for i in has_offer:
    with open(f'../../data/separated_by_email/{i}.json') as json_file:
        x = json.load(json_file)
        content = x['X-TIKA:content'].replace('\n', '').split('.')
        for sentence in content: 
            if 'service' in sentence.lower():
                print(f'SERVICE: {sentence}')
                print('-----------------------------------------')
            if 'offer' in sentence.lower():
                print(f'OFFER: {sentence}')
                print('-----------------------------------------')
            elif 'reward' in sentence.lower():
                print(f'REWARD: {sentence}')
                print('-----------------------------------------')
                
#         tokens = nlp(content)
#         for sentence in tokens.sents:
#             print(sentence)

OFFER: OfferMr George MboroConfidential Tel :++ 874-76-253-5870 Confidential Fax:++ 874-76-253-5871Office Tel ++ 27-83-49-92446 Dear Sir, STRICTLY CONFIDENTIAL & URGENT
-----------------------------------------
SERVICE:  More so, we are handicapped in the circumstances, as the South Africa Civil Service Code of Conduct does not allow us to operate offshore account hence your importance in the whole transaction
-----------------------------------------
SERVICE:  I want to assure you that myself and my partners are in a position to make the payment of this claim possible provided you can give us a very strong assurance and guarantee that our share will be secured and Please remember to treat this matter as very confidential matter; because we will not comprehend with any form of exposure as we are still in active Government Service and remember once again that time is of the essence in this business
-----------------------------------------
REWARD: IF YOU ARE WILLING TO ASSIST US IN RECE

In [65]:
# output
iv_list

[{'index': 1, 'has_offer': 'no'},
 {'index': 2, 'has_offer': 'no'},
 {'index': 3, 'has_offer': 'no'},
 {'index': 4, 'has_offer': 'no'},
 {'index': 5, 'has_offer': 'no'},
 {'index': 6, 'has_offer': 'no'},
 {'index': 7, 'has_offer': 'no'},
 {'index': 8, 'has_offer': 'no'},
 {'index': 9, 'has_offer': 'no'},
 {'index': 10, 'has_offer': 'no'},
 {'index': 11, 'has_offer': 'no'},
 {'index': 12, 'has_offer': 'no'},
 {'index': 13, 'has_offer': 'no'},
 {'index': 14, 'has_offer': 'no'},
 {'index': 15, 'has_offer': 'no'},
 {'index': 16, 'has_offer': 'no'},
 {'index': 17, 'has_offer': 'no'},
 {'index': 18, 'has_offer': 'no'},
 {'index': 19, 'has_offer': 'no'},
 {'index': 20, 'has_offer': 'no'},
 {'index': 21, 'has_offer': 'no'},
 {'index': 22, 'has_offer': 'no'},
 {'index': 23, 'has_offer': 'no'},
 {'index': 24, 'has_offer': 'service'},
 {'index': 25, 'has_offer': 'no'},
 {'index': 26, 'has_offer': 'no'},
 {'index': 27, 'has_offer': 'no'},
 {'index': 28, 'has_offer': 'no'},
 {'index': 29, 'has_offe

In [68]:
for i in tqdm(range(0, 4029)):
    requests.patch(f'https://dsci-550-default-rtdb.firebaseio.com/assignment-1/task-5-additional-features/{i+1}.json', json=iv_list[i])

  0%|          | 0/4029 [00:00<?, ?it/s]

# x.

In [54]:
def get_ip_info():
    ip_list = []
    
    for i in tqdm(range(1, 4030)):
        ip_dict = {'index': i, 'ip': 'unknown'}
        
        with open(f'../../data/separated_by_email/{i}.json') as json_file:
            x = json.load(json_file)
        try:
            ip_original = x['MboxParser-x-originating-ip']
        except:
            try: 
                ip_original = x['MboxParser-x-sender-ip']
            except:
                ip_list.append(ip_dict)
                continue

        ip_address = re.search(r'\d+\.\d+\.\d+\.\d+', ip_original).group()
        url = f'https://scamalytics.com/ip/{ip_address}'
        content = requests.get(url)
        soup = BeautifulSoup(content.content, features='html.parser')
        content.close()

        risk = soup.find('pre').text
        risk_dict = json.loads(risk)
        ip_dict['ip'] = {'ip_address': risk_dict['ip'], 'score': risk_dict['score'], 'risk': risk_dict['risk']}
        ip_list.append(ip_dict)
        
    return ip_list

In [55]:
ip_list = get_ip_info()

100%|██████████| 4029/4029 [07:10<00:00,  9.35it/s]


In [56]:
# output
ip_list

[{'index': 1, 'ip': 'unknown'},
 {'index': 2, 'ip': 'unknown'},
 {'index': 3, 'ip': 'unknown'},
 {'index': 4, 'ip': 'unknown'},
 {'index': 5, 'ip': 'unknown'},
 {'index': 6, 'ip': 'unknown'},
 {'index': 7,
  'ip': {'ip_address': '208.14.9.28', 'score': '8', 'risk': 'low'}},
 {'index': 8, 'ip': 'unknown'},
 {'index': 9, 'ip': 'unknown'},
 {'index': 10,
  'ip': {'ip_address': '208.14.9.28', 'score': '8', 'risk': 'low'}},
 {'index': 11, 'ip': 'unknown'},
 {'index': 12, 'ip': 'unknown'},
 {'index': 13, 'ip': 'unknown'},
 {'index': 14, 'ip': 'unknown'},
 {'index': 15, 'ip': 'unknown'},
 {'index': 16, 'ip': 'unknown'},
 {'index': 17, 'ip': 'unknown'},
 {'index': 18, 'ip': 'unknown'},
 {'index': 19, 'ip': 'unknown'},
 {'index': 20, 'ip': 'unknown'},
 {'index': 21, 'ip': 'unknown'},
 {'index': 22, 'ip': 'unknown'},
 {'index': 23, 'ip': 'unknown'},
 {'index': 24, 'ip': 'unknown'},
 {'index': 25, 'ip': 'unknown'},
 {'index': 26, 'ip': 'unknown'},
 {'index': 27, 'ip': 'unknown'},
 {'index': 28, '

In [60]:
for i in tqdm(range(0, 4029)):
    requests.patch(f'https://dsci-550-default-rtdb.firebaseio.com/assignment-1/task-5-additional-features/{i+1}.json', json=ip_list[i])

100%|██████████| 4029/4029 [17:49<00:00,  3.77it/s]
